# Proyecto: Particionamiento de información

En la presente práctica veremos cómo:

* Particionar un RDD en distintos archivos y guardarlos en archivos externos.

* Cargar un archivo .csv en un RDD k-particionado

* Guardar los elementos de una partición en archivos de texto externos

* Recolectar archivos externos de una partición en un solo RDD 

* Recolectar archivos externos de una partición en un solo Data Frame 

Algunas funciones que veremos son:

`.rdd.getNumPartitions()` Regresa el número de particiones de un Data Frame

`.getNumPartitions()` Regresa el número de particiones de un RDD


In [1]:
# Importamos librerias para crear sesiones de Spark 
from pyspark.sql import SparkSession

# Construimos una sesión de Spark
spark = SparkSession.builder.appName('Particionado').master('local[5]').getOrCreate()

# 'local[5]' <-- indica que haremos un particionado de tamaño 5

## Particionamiento de Data Frames:

In [2]:
# Creamos un Data Frame:
df = spark.range(0,20)

In [3]:
# Visualizamos el Data Frame:
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



In [4]:
# Obtenemos el número de particiones:
df.rdd.getNumPartitions()

5

## Particionamiento de RDDs

In [5]:
# Creamos otro RDD:

# Numero de particiones
k=10

rdd1 = spark.sparkContext.parallelize( (0,20) , k )

# ( (0,20) , k ) <-- hace k particiones del intervalo (0,20)

In [6]:
# Obtenemos el número de particiones:
rdd1.getNumPartitions()

10

### Cargamos un archivo .csv en un RDD k-particionado

In [7]:
# Numero de particiones
k=10

# Cargamos archivo .csv en un RDD k-particionado
rddDesdeArchivo = spark.sparkContext.textFile( './Data/deporte.csv' , k )

In [8]:
# Visualizamos los primeros 20 registros del RDD:
rddDesdeArchivo.take(20)

['deporte_id,deporte',
 '1,Basketball',
 '2,Judo',
 '3,Football',
 '4,Tug-Of-War',
 '5,Speed Skating',
 '6,Cross Country Skiing',
 '7,Athletics',
 '8,Ice Hockey',
 '9,Swimming',
 '10,Badminton',
 '11,Sailing',
 '12,Biathlon',
 '13,Gymnastics',
 '14,Art Competitions',
 '15,Alpine Skiing',
 '16,Handball',
 '17,Weightlifting',
 '18,Wrestling',
 '19,Luge']

In [9]:
# Obtenemos el número de particiones del RDD:
rddDesdeArchivo.getNumPartitions()

10

In [10]:
# Obtenemos el número de registros del RDD:
rddDesdeArchivo.countApprox(20)

67

## Guardando elementos de una partición en archivos de texto

In [11]:
# En caso de error, debemos eliminar el directorio 'salidatexto'
!rm -r ./Data/salidatexto

In [12]:
# Guardamos los elementos de la partición como archivos de texto en la carpeta 'salidatexto'
rddDesdeArchivo.saveAsTextFile('./Data/salidatexto')

In [13]:
# Vemos los archivos que se generaron en la ruta: ./Data/salidatexto
!ls ./Data/salidatexto

part-00000  part-00002	part-00004  part-00006	part-00008  _SUCCESS
part-00001  part-00003	part-00005  part-00007	part-00009


Los archivos 'part-0000x' corresponden a los elementos de la partición del RDD

Si el contenido del archivo '_SUCCESS' es vacío, significa que la operación se realizó correctamente

In [14]:
# Vemos el contenido del archivo '_SUCCESS'
!cat ./Data/salidatexto/_SUCCESS

In [15]:
# Vemos el contenido del archivo 'part-00000'
!cat ./Data/salidatexto/part-00000

deporte_id,deporte
1,Basketball
2,Judo
3,Football
4,Tug-Of-War
5,Speed Skating
6,Cross Country Skiing


In [16]:
# Vemos el contenido del archivo 'part-00001'
!cat ./Data/salidatexto/part-00001

7,Athletics
8,Ice Hockey
9,Swimming
10,Badminton
11,Sailing
12,Biathlon
13,Gymnastics
14,Art Competitions


## Recolección de elementos de una partición en un RDD:

In [17]:
# Creamos un RDD con todos los archivos de la partición:
rdd = spark.sparkContext.wholeTextFiles('./Data/salidatexto/*')

In [18]:
# Generamos una lista con los elementos del RDD
lista_archivos = rdd.mapValues(lambda x : x.split() ).collect()

In [19]:
# Vemos los elementos de 'lista_archivos'
lista_archivos

[('file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00000',
  ['deporte_id,deporte',
   '1,Basketball',
   '2,Judo',
   '3,Football',
   '4,Tug-Of-War',
   '5,Speed',
   'Skating',
   '6,Cross',
   'Country',
   'Skiing']),
 ('file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00001',
  ['7,Athletics',
   '8,Ice',
   'Hockey',
   '9,Swimming',
   '10,Badminton',
   '11,Sailing',
   '12,Biathlon',
   '13,Gymnastics',
   '14,Art',
   'Competitions']),
 ('file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00002',
  ['15,Alpine',
   'Skiing',
   '16,Handball',
   '17,Weightlifting',
   '18,Wrestling',
   '19,Luge',
   '20,Water',
   'Polo']),
 ('file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00003',
  ['21,Hockey',
   '22,Rowing',
   '23,Bobsle

In [20]:
# Notemos que la primer componente [0] de cada elemento de la lista 'lista_archivos'
# corresponde a la ruta de un elemento de la partición:
print(lista_archivos[0][0])
print(lista_archivos[1][0])
print(lista_archivos[2][0])

file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00000
file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00001
file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00002


In [21]:
# Guardamos las rutas (elemento l[0] de la 'lista' aterior) de cada elemento de la partición:
lista_dir = [ l[0] for l in lista_archivos ]

# Ordenamos las rutas:
lista_dir.sort()

In [22]:
# Vemos el contenido de la 'lista'
lista_dir

['file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00000',
 'file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00001',
 'file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00002',
 'file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00003',
 'file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00004',
 'file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00005',
 'file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00006',
 'file:/home/marck/Escritorio/Platzi_Cursos_Nuevos/C18_Fundamentos_Spark_Big_Data/Practicas/Data/salidatexto/part-00007',
 'file:/home/marck/Escri

In [23]:
# Recolectamos los archivos externos en un solo RDD:

# Recordemos que 'k' es el número de particiones
rdd2 = spark.sparkContext.textFile(','.join(lista_dir) , k).map(lambda l: l.split(','))

In [24]:
rdd2.take(20)

[['deporte_id', 'deporte'],
 ['1', 'Basketball'],
 ['2', 'Judo'],
 ['3', 'Football'],
 ['4', 'Tug-Of-War'],
 ['5', 'Speed Skating'],
 ['6', 'Cross Country Skiing'],
 ['7', 'Athletics'],
 ['8', 'Ice Hockey'],
 ['9', 'Swimming'],
 ['10', 'Badminton'],
 ['11', 'Sailing'],
 ['12', 'Biathlon'],
 ['13', 'Gymnastics'],
 ['14', 'Art Competitions'],
 ['15', 'Alpine Skiing'],
 ['16', 'Handball'],
 ['17', 'Weightlifting'],
 ['18', 'Wrestling'],
 ['19', 'Luge']]

## Recolección de particiones (en archivos externos) en un Data Frame:

In [25]:
# Cargamos librerias para crear el schema del DataFrame
from pyspark.sql.types import StructType, StructField

# Cargamos los tipos de datos que usaremos para crear las columnas de los dataframes:
from pyspark.sql.types import  IntegerType, StringType, FloatType

rdd_schema = StructType([
    StructField('deporte_id', IntegerType(), False),
    StructField('deporte', StringType(), False)
])

# Creamos el Data Frame:
DF = spark.read.csv('./Data/salidatexto',rdd_schema)


In [26]:
DF.sort(DF['deporte_id']).show(20)

+----------+--------------------+
|deporte_id|             deporte|
+----------+--------------------+
|      null|             deporte|
|         1|          Basketball|
|         2|                Judo|
|         3|            Football|
|         4|          Tug-Of-War|
|         5|       Speed Skating|
|         6|Cross Country Skiing|
|         7|           Athletics|
|         8|          Ice Hockey|
|         9|            Swimming|
|        10|           Badminton|
|        11|             Sailing|
|        12|            Biathlon|
|        13|          Gymnastics|
|        14|    Art Competitions|
|        15|       Alpine Skiing|
|        16|            Handball|
|        17|       Weightlifting|
|        18|           Wrestling|
|        19|                Luge|
+----------+--------------------+
only showing top 20 rows



In [27]:
# Debido a que el primer registro de la partición 'part-00000' contiene encabezado, 
# este se cuenta como un registro, lo cual es incorrecto así que lo eliminamos con 'filter'
DF.sort(DF['deporte_id']).filter( (DF['deporte'] != 'deporte') ).show(20)

+----------+--------------------+
|deporte_id|             deporte|
+----------+--------------------+
|         1|          Basketball|
|         2|                Judo|
|         3|            Football|
|         4|          Tug-Of-War|
|         5|       Speed Skating|
|         6|Cross Country Skiing|
|         7|           Athletics|
|         8|          Ice Hockey|
|         9|            Swimming|
|        10|           Badminton|
|        11|             Sailing|
|        12|            Biathlon|
|        13|          Gymnastics|
|        14|    Art Competitions|
|        15|       Alpine Skiing|
|        16|            Handball|
|        17|       Weightlifting|
|        18|           Wrestling|
|        19|                Luge|
|        20|          Water Polo|
+----------+--------------------+
only showing top 20 rows



In [28]:
# Cerramos sesión para liberar memoria:
spark.stop()